# HyDE Query Transform

## Download Data

In [1]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2025-07-08 11:44:00--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 75042 (73K) [text/plain]
저장 위치: `data/paul_graham/paul_graham_essay.txt'

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    / 0.007s   

2025-07-08 11:44:00 (9.87 MB/s) - `data/paul_graham/paul_graham_essay.txt' 저장함 [75042/75042]



## Load documents, build the VectorStoreIndex

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="/media/user/datadisk2/Embedding_models/bge-m3",
    device="cuda",
    target_devices="cuda:0"
)

Settings.llm = HuggingFaceLLM(
    model_name="/media/user/datadisk2/LLM_models/Qwen2.5-14B-Instruct-1M",
    tokenizer_name="/media/user/datadisk2/LLM_models/Qwen2.5-14B-Instruct-1M",
    device_map="auto",
    max_new_tokens=2048
)

/home/user/anaconda3/envs/2hg_llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: /media/user/datadisk2/Embedding_models/bge-m3
Load pretrained SentenceTransformer: /media/user/datadisk2/Embedding_models/bge-m3


Loading checkpoint shards: 100%|██████████| 8/8 [00:19<00:00,  2.45s/it]


In [4]:
# load documents
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(documents)

## Example: HyDE improves specific temporal queries

In [5]:
query_str = "what did paul graham do after going to RISD"

First, we query without transformation: The same query string is used for embedding lookup and also summarization.

In [6]:
query_engine = index.as_query_engine(llm=None)
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b> After Paul Graham returned to RISD in the fall of 1992, he continued his studies in the painting department but found the experience disappointing. He observed that the painting department at RISD was post-rigorous, meaning it focused more on encouraging students to develop a signature style rather than providing rigorous training in artistic techniques. This observation was further reinforced when Graham applied to study at the Accademia di Belle Arti in Florence, Italy, where he encountered a similar lack of structured education. Despite passing the entrance exam by writing an essay on Cézanne, Graham noted that the students and faculty had reached an agreement where neither party expected much from the other, maintaining the appearance of a traditional 19th-century atelier while lacking substantive teaching.

During his time at RISD, Graham also engaged in freelance work for the group that handled customer projects at Interleaf, which helped him financially survive the next several years. Additionally, he kept working on his book "On Lisp," which he had already secured a contract to publish. This period marked a time of exploration and learning, where Graham balanced his artistic pursuits with his passion for Lisp hacking and his professional obligations.</b>

Now, we use `HyDEQueryTransform` to generate a hypothetical document and use it for embedding lookup.

In [7]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))

 After attending RISD, Paul Graham took a job at a company called Interleaf, which made software for creating documents, similar to Microsoft Word. He worked there for a year while secretly working on his book "On Lisp," for which he had secured a publishing contract. During this period, he earned significant money, allowing him to save enough to return to RISD and pay off his college loans. After leaving Interleaf, he arranged to do freelance work for the customer project group, which sustained him financially over the next several years. During one of his visits back to Interleaf, he heard about a new thing called HTML, which later played a significant role in his career. In the fall of 1992, he returned to RISD to continue his studies, where he found the painting department to be more about expressing oneself and developing a signature style rather than rigorous training in art. This experience contrasted with his earlier impression of RISD as a place for serious art education, and he realized that art schools like RISD did not necessarily prepare students for the rigors of professional art practice.

In this example, `HyDE` improves output quality significantly, by hallucinating accurately what Paul Graham did afer RIS (see below), and thus improving the embedding quality, and final output.

In [8]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [9]:
hyde_doc

'Paul Graham, an accomplished photographer and writer, attended the Rhode Island School of Design (RISD) from 1983 to 1987. After graduating from RISD, he moved to New York City in 1987, where he immersed himself in the city\'s vibrant art scene. He worked at Aperture Foundation, a non-profit organization dedicated to advancing photography as a fine art form, from 1988 to 1990. During his time there, he edited and published several books on photography. In 1990, Graham embarked on a series of photographic projects that would define his career. He traveled extensively, capturing the essence of different cultures and landscapes through his lens. His first major project was "A1," a black-and-white series documenting the M25 motorway around London. This work showcased his unique approach to street photography and laid the foundation for his future projects. Throughout the 1990s, Graham continued to produce thought-provoking and critically acclaimed works, such as "Beyond the Pale" and "Pro

## Failure case 1: HyDE may mislead when query can be mis-interpreted without context.

In [10]:
query_str = "What is Bel?"

Querying without transformation yields reasonable answer

In [11]:
response = query_engine.query(query_str)
display(Markdown(f"{response}"))

 Bel is a new Lisp language created by Paul Graham, developed from March 26, 2015, to October 12, 2019. Inspired by John McCarthy's original Lisp, Bel aims to be a complete programming language based on McCarthy's axiomatic approach, where each modification is a "discoveredness-preserving transformation." The language was written in itself within the Arc programming language, employing various hacks to achieve self-interpreting capabilities. The development process was challenging due to the intricate nature of creating an interpreter written in itself, leading to difficulties in understanding and debugging the code. Graham dedicated significant effort to Bel, even prioritizing its completion over writing essays, and found the experience both difficult and rewarding. He recalls moments of intense focus and satisfaction, including solving complex problems while spending time with his children. The majority of Bel's development occurred in England, where the Graham family moved temporarily to provide their children with international experiences. After four years of intensive work, Bel was finally completed in the fall of 2019.

Querying with `HyDEQeuryTransform` results in nonsense

In [12]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"{response}"))

 Bel is a new Lisp created by Paul Graham, designed to follow McCarthy's axiomatic approach in defining a complete programming language. Graham developed Bel in itself using Arc, facing significant challenges due to the intricate nature of creating an interpreter written in its own language. The project spanned four years, from March 26, 2015, to October 12, 2019, during which Graham dedicated himself entirely to the task, avoiding distractions such as writing essays. This intense focus led to the misconception that he was inactive, despite his rigorous work on Bel. The complexity of the project required Graham to maintain a deep understanding of the code, which became increasingly difficult as the project progressed. However, he found moments of satisfaction and clarity, even in challenging times, and completed Bel in the fall of 2019. During the development period, the Graham family moved to England, where they intended to stay for a year but ended up settling permanently. The creation of Bel was both challenging and fulfilling for Graham, representing a significant achievement in his career.

In this example, `HyDE` mis-interprets Bel without document context (see below), resulting in a acompletely unrelated embedding string and poor retrieval outcome.

In [13]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]
hyde_doc

'Bel, also known as Baal or Belus, was a Semitic god worshipped in the ancient Near East. He was often associated with storms and agriculture, and was considered the lord of life and death. In some traditions, he was also linked to the sun and was seen as a protector of cities and nations. The worship of Bel spread throughout the region, and he became one of the most important deities in Mesopotamian religion. Temples dedicated to Bel were built in various cities, including Babylon, where he was considered the chief deity. Over time, Bel\'s attributes and role evolved, and he came to be identified with other gods such as Marduk and El. Despite this, Bel remained an influential figure in the religious and cultural landscape of the ancient Near East for centuries.\n"""'

## Failure cate 2: HyDE may bias open-ended queries

In [14]:
query_str = "What would the author say about art vs. engineering?"

Querying without transformation yields a reasonable answer

In [15]:
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b> Based on the additional context provided, the author's perspective on art versus engineering remains consistent with the initial analysis but gains depth through personal experiences and motivations.

Regarding engineering, particularly in the context of computer science, the author continues to express dissatisfaction with the transient nature of systems work. He notes that programs written today will likely be obsolete within a couple of decades. Even if they were highly regarded in their time, they would seem feeble and outdated to future generations. The author mentions that systems work doesn't last, and uses the example of surplus Xerox Dandelions that were once cutting-edge but are now considered slow and impractical by current standards. This leads him to conclude that while systems work can be exciting, it lacks permanence and impact over the long term.

On the other hand, the author finds great appeal in art, especially painting. He discovers that paintings can endure for centuries and retain their value and relevance. For instance, he reflects on visiting a museum and realizing that the painting on the wall is something tangible and lasting, unlike the ephemeral nature of software. Furthermore, he appreciates the potential for artistic independence, where artists don't have bosses or rely on external funding, allowing for greater freedom and creativity. The author's newfound passion for art is highlighted by his decision to take art classes at Harvard during his PhD program in computer science, despite knowing that it might complicate his academic career. He describes the idea of making art as almost miraculous, emphasizing his curiosity and fascination with the process of creating art. He likens painting still lives to closely observing the world, noticing details that are usually overlooked in everyday life. This deep engagement with the subject matter allows for a more nuanced and realistic portrayal, even surpassing photographic accuracy in some ways. His interest in art stems from a desire to understand and capture what he sees, rather than just replicating it superficially.

The additional context reveals the author's internal struggle between his commitment to his PhD program in computer science and his burgeoning passion for art. He feels trapped in a situation where he cannot pursue both interests fully, which adds a layer of complexity to his views on the two fields. Despite the challenges, the author remains dedicated to both pursuits, finding inspiration and fulfillment in each. Ultimately, he values the enduring nature and creative independence of art more than the transient nature of engineering, but he also recognizes the excitement and intellectual challenge that engineering provides. This duality reflects the multifaceted nature of his identity and his ongoing quest to balance his professional and artistic aspirations.</b>

In [16]:
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b> Based on the additional context provided, the author's perspective on art versus engineering deepens, reflecting both personal experiences and philosophical musings. The author contrasts art and engineering (or systems work) in terms of longevity, the satisfaction derived from creation, and the personal journey involved in pursuing each discipline.

Regarding engineering or systems work, the author expresses dissatisfaction with its transient nature. Programs and software created today may become obsolete within a couple of decades, and even if they are recognized in historical contexts, their practical utility diminishes over time. The author laments that despite the initial brilliance of these works, they would seem "very feeble" to future generations who lack an understanding of their historical significance. This transient quality is exemplified by the fate of surplus Xerox Dandelion computers, which became outdated and were discarded.

In contrast, the author finds art, particularly painting, to be a medium that transcends time. Unlike engineering projects, paintings can retain their value and relevance for centuries, as evidenced by the enduring presence of ancient masterpieces in museums. This permanence appeals to the author, who seeks to create lasting contributions. Moreover, the author appreciates the independence and freedom associated with being an artist. Artists can operate without bosses or the need for research funding, allowing for a greater degree of autonomy compared to the structured environment of engineering or academia.

The author's personal journey adds depth to this comparison. Initially, the author struggled with the idea of becoming an artist, seeing artists as a different species, either living long ago or as mysterious geniuses. However, after taking art classes at Harvard, the author began to see the possibility of creating art as something achievable. This realization opened up a new path, leading the author to pursue both art and computer science concurrently. The author's experience in grad school, where they worked on multiple projects outside their thesis, further highlights the creative freedom and autonomy that art offers, contrasting with the structured environment of academia.

In summary, while the author acknowledges the intellectual rigor and admiration for theory in engineering, they find art to be more fulfilling due to its potential for lasting impact and the creative freedom it offers. Art, in contrast to engineering, allows for the creation of timeless pieces that continue to captivate and inspire across generations. The author's personal journey underscores the transformative power of embracing art, even in the midst of academic pursuits.</b>

참고  
- [HyDE Query Transform](https://docs.llamaindex.ai/en/stable/examples/query_transformations/HyDEQueryTransformDemo/)  
- [HyDE 설명](https://devocean.sk.com/community/detail.do?ID=165999&boardType=DEVOCEAN_STUDY&page=1)